In [ ]:
import numpy as np
import scipy.linalg as la

# shifted matrix B = (A-delta*I)
# This will return one A's eigenvals from one eigenvals of B
def get_eigval_Shifted(eigval_B, delta):
  return eigval_B + delta

# shifted inverse matrix B = (A-delta*I)^-1
# This will return one A's eigenvals from one eigenvals of B
def get_eigval_Shift_inverse(eigval_B, delta):
  return (1/eigval_B) + delta


# C = (A^alpha + delta*B)
# this function returns eigval of C based on eigval of A and B and delta
def get_eigval_combined_1(eigval_A, eigval_B, delta,alpha):
  return (eigval_A**alpha + delta*eigval_B)

# C = (A + delta*B)^k
# this function returns eigval of C based on eigval of A and B and delta
def get_eigval_combined_1(eigval_A, eigval_B, delta,k):
  return (eigval_A + delta*eigval_B)**k

# x is the eigenvector of A
# returns the eigenval of A associate with x
def eigvec_to_eigval(A,x):
  return (x.T @ A @ x) / (x.T @ x)

# A must be a 1d array contains diag values
def converge_rate(A, delta=0):
  mag = np.sort(np.abs(A))
  rate_npi = mag[-2]/mag[-1]
  rate_inv = mag[0]/mag[1]
  print("=== Converge rate: ===")
  print("Converge rate (Normal): \n\t" + str(rate_npi))
  print("Converge rate (Inverse): \n\t" + str(rate_inv))

  if(delta!=0):
    diff = np.abs(A-delta)
    indices = np.argsort(diff)
    rate_shift = diff[indices[0]]/diff[indices[1]]
    print("Converge rate (Shifted): \n\t" + str(rate_shift))

def get_eig_from_diag(X,D, delta = 0, norm = 2):
  vals = D.diagonal()
  npi = X[:,np.argmax(vals)]
  inv = X[:,np.argmin(vals)]
  npi /= la.norm(npi,norm)
  inv /= la.norm(inv,norm)
  print("Eigvec for Norm Power Iteration   : \n\t" + str(npi))
  print("Eigvec for Inversed Norm Iteration: \n\t" + str(inv))
  if(delta!=0):
    inx = np.nanargmin(np.abs(vals-delta))
    sinv = X[:,inx]
    sinv /= la.norm(sinv,norm)
    print("Eigvec for Shifted inv Iteration: \n\t" + str(sinv))


# This returns the biggest eigenvalue with associate eigenvector of A
def norm_power_iteration(A: np.ndarray, tol=10**-12, iteration=0, x0=None):
  if (type(x0)==type(None)): 
    x0 = np.random.rand(A.shape[0])
  P,L,U = la.lu(A)
  x_npi = np.copy(x0)/la.norm(x0,2)
  count = 0
  while True:
    count += 1
    y_npl = A @ x_npi
    y_npl /= la.norm(y_npl,2)
    if (la.norm(y_npl-x_npi,2)<=tol or (iteration!=0 and count>=iteration)): break
    x_npi = y_npl
  eigvec = x_npi
  eigval = eigvec_to_eigval(A,x_npi)
  return eigvec, eigval, count
  
# This returns the smallest eigenvalue with associate eigenvector of A
def inverse_power_iteration(A: np.ndarray, tol=10**-12, iteration = 0, x0=None):
  if (type(x0)==type(None)): 
    x0 = np.random.rand(A.shape[0])
  P,L,U = la.lu(A)
  x_inv = np.copy(x0)/la.norm(x0,2)
  count = 0
  # inversed
  while True:
    count += 1
    y_inv = la.solve_triangular(L, np.dot(P.T,x_inv), lower=True)
    y_inv = la.solve_triangular(U, y_inv)
    y_inv /= la.norm(y_inv,2)
    if (la.norm(x_inv-y_inv,2)<=tol or (iteration!=0 and count>=iteration)): break
    x_inv = y_inv
  eigvec = x_inv
  eigval = eigvec_to_eigval(A,x_inv)
  return eigvec, eigval, count

# shifted matrix B = (A-delta*I)
def shifted_inverse_power_iteration(A: np.ndarray, delta:float, tol=10**-12, iteration = 0, x0=None):
  if (type(x0)==type(None)): 
    x0 = np.random.rand(A.shape[0])
  B = (A-delta*np.identity(A.shape[0]))
  P,L,U = la.lu(B)
  x_shift = np.copy(x0)/la.norm(x0,2)
  count = 0
  # shifted inverse power iteration
  while True:
    count += 1
    y_shift = la.solve_triangular(L, np.dot(P.T,x_shift), lower=True)
    y_shift = la.solve_triangular(U, y_shift)
    y_shift /= la.norm(y_shift,2)
    if (la.norm(x_shift-y_shift,2)<=tol or (iteration!=0 and count>=iteration)): break
    x_shift = y_shift
  eigvec = x_shift
  eigval = eigvec_to_eigval(A,x_shift)
  return eigvec, eigval, count

In [ ]:
# Inverse Shift Iteration
get_eigval_Shift_inverse(0.5, 2)

In [ ]:
# Result of Normalized Power, Inverse, and Shifted Inverse Iteration
X = np.array([[0,12,14],[17,17,18],[1,2,16]],dtype=np.float64)
D = np.array([[9,0,0],[0,6,0],[0,0,3]],dtype=np.float64)
A = X @ D @ la.inv(X)
get_eig_from_diag(X,D,delta = 6.3, norm = np.inf)

In [ ]:
# Choose Fastest Convergence
A= np.array([2,11,88])
converge_rate(A,delta=91)

In [ ]:
get_eigval_Shift_inverse(10, -7)

In [ ]:
X = np.array([3,7,13,4,19,11,12,8,2],dtype=np.float64).reshape((3,3))
D = np.array([7,0,0,0,2,0,0,0,8],dtype=np.float64).reshape((3,3))

get_eig_from_diag(X,D,norm=np.inf)


In [ ]:
X = np.array([5,0,8,0,17,6,7,9,14],dtype=np.float64).reshape((3,3))
D = np.array([1,0,0,0,3,0,0,0,4],dtype=np.float64).reshape((3,3))

get_eig_from_diag(X,D,delta=3.7,norm=1)
